# **数据预处理**
* 深度学习第一步，肯定是处理数据

In [ ]:
# 解压数据集
!rm -r ~/work/PaddleDetection/dataset/insects
!unzip -q -d ~/work/PaddleDetection/dataset/ ~/data/data19638/insects.zip

# 数据集预处理
# 1.将原训练集中的‘Linnaeus’与‘linnaeus’统一为‘Linnaeus’
# 2.打乱原来的训练和验证集数据，重新分割训练和验证集
# 3.将原数据转换成PaddleDetection所支持的VOC数据集
!python ~/work/PaddleDetection/tools/preprocess.py

# 加入测试集标注文件
!rm -r ~/work/test_annotations
!unzip -q ~/work/test_annotations.zip -d  ~/work
!mkdir ~/work/PaddleDetection/dataset/insects/test/annotations
!cp -r ~/work/test_annotations/xmls ~/work/PaddleDetection/dataset/insects/test/annotations
!cp ~/work/test_annotations/test.txt ~/work/PaddleDetection/dataset/insects

预处理完成！


# **模型训练**
* 可通过右侧可视化选项,来启动VisualDL观察可视化的训练数据

In [9]:
%cd ~/work/PaddleDetection

!python -u tools/train.py \
    -c configs/insects/ppyolo_2x.yml \
    --eval \
    --use_vdl True \
    --vdl_log_dir ~/log/insects

/home/aistudio/work/PaddleDetection
2024-01-12 12:50:33,545-INFO: If regularizer of a Parameter has been set by 'fluid.ParamAttr' or 'fluid.WeightNormParamAttr' already. The Regularization[L2Decay, regularization_coeff=0.000500] in Optimizer will not take effect, and it will only be applied to other Parameters!
2024-01-12 12:50:37,250-INFO: places would be ommited when DataLoader is not iterable
W0112 12:50:37.319314  2912 device_context.cc:252] Please NOTE: device: 0, CUDA Capability: 70, Driver API Version: 12.0, Runtime API Version: 9.0
W0112 12:50:37.324162  2912 device_context.cc:260] device: 0, cuDNN Version: 7.6.
2024-01-12 12:50:42,800-WARNING: variable yolo_output.2.conv.weights not used
2024-01-12 12:50:42,800-WARNING: variable yolo_output.1.conv.bias not used
2024-01-12 12:50:42,800-WARNING: variable yolo_output.0.conv.weights not used
2024-01-12 12:50:42,800-WARNING: variable yolo_output.2.conv.bias not used
2024-01-12 12:50:42,800-WARNING: variable yolo_output.1.conv.weigh

# **模型验证**
* 使用不同的配置文件，对验证集和测试集的效果进行评估

In [10]:
# 模型评估（验证集）
%cd ~/work/PaddleDetection

!python -u tools/eval.py -c configs/insects/ppyolo_2x.yml

/home/aistudio/work/PaddleDetection
2024-01-12 15:15:56,504-INFO: places would be ommited when DataLoader is not iterable
W0112 15:15:56.554306 15379 device_context.cc:252] Please NOTE: device: 0, CUDA Capability: 70, Driver API Version: 12.0, Runtime API Version: 9.0
W0112 15:15:56.560933 15379 device_context.cc:260] device: 0, cuDNN Version: 7.6.
2024-01-12 15:16:11,765-INFO: Test iter 0
2024-01-12 15:16:16,562-INFO: Test finish iter 11
2024-01-12 15:16:16,563-INFO: Total number of images: 245, inference time: 18.99610110236985 fps.
2024-01-12 15:16:16,563-INFO: Start evaluate...
2024-01-12 15:16:17,981-INFO: Accumulating evaluatation results...
2024-01-12 15:16:18,164-INFO: mAP(0.50, 11point) = 81.36
FATAL: exception not rethrown
W0112 15:16:18.366008 15534 init.cc:226] Warning: PaddlePaddle catches a failure signal, it may not work properly
W0112 15:16:18.366058 15534 init.cc:228] You could check whether you killed PaddlePaddle thread/process accidentally or report the case to Padd

In [11]:
# 评估模型（测试集）
%cd ~/work/PaddleDetection

!python -u tools/eval.py -c configs/insects/ppyolo_2x_test.yml

/home/aistudio/work/PaddleDetection
2024-01-12 15:16:22,614-INFO: places would be ommited when DataLoader is not iterable
W0112 15:16:22.678679 15564 device_context.cc:252] Please NOTE: device: 0, CUDA Capability: 70, Driver API Version: 12.0, Runtime API Version: 9.0
W0112 15:16:22.683676 15564 device_context.cc:260] device: 0, cuDNN Version: 7.6.
2024-01-12 15:16:35,082-INFO: Test iter 0
2024-01-12 15:16:40,270-INFO: Test finish iter 11
2024-01-12 15:16:40,270-INFO: Total number of images: 245, inference time: 19.220053049060336 fps.
2024-01-12 15:16:40,271-INFO: Start evaluate...
2024-01-12 15:16:41,786-INFO: Accumulating evaluatation results...
2024-01-12 15:16:41,898-INFO: mAP(0.50, 11point) = 90.78
terminate called without an active exception
terminate called recursively
W0112 15:16:42.129732 15660 init.cc:226] Warning: PaddlePaddle catches a failure signal, it may not work properly
W0112 15:16:42.129791 15660 init.cc:228] You could check whether you killed PaddlePaddle thread/pr

# **导出推理模型**

In [12]:
%cd ~/work/PaddleDetection
!python tools/export_model.py \
    -c configs/insects/ppyolo_2x.yml \
    --output_dir=./inference_model

/home/aistudio/work/PaddleDetection
2024-01-12 15:18:08,401-INFO: save_inference_model pruned unused feed variables im_id
2024-01-12 15:18:08,401-INFO: Export inference model to ./inference_model/ppyolo_2x, input: ['image', 'im_size'], output: ['matrix_nms_0.tmp_0']...
2024-01-12 15:18:10,562-INFO: Load categories from dataset/insects/label_list.txt
2024-01-12 15:18:10,564-INFO: Export inference config file to ./inference_model/ppyolo_2x/infer_cfg.yml


# **模型测试**
* 可视化输出检测结果，测试模型真实效果

In [13]:
%cd ~/work/PaddleDetection
!python deploy/python/infer.py \
    --model_dir inference_model/ppyolo_2x \
    --image_file dataset/insects/test/images \
    --output_dir ~/data/output \
    --use_gpu True \
    --threshold 0.5